In [20]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3
from eth_account import Account
from web3.middleware import geth_poa_middleware

In [21]:

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")


In [22]:

# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE
from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
 
 


In [23]:
# Create a function called `derive_wallets`
def derive_wallets(mnemonic, coin, numderive):
    
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys


In [24]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {"eth", "btc-test", "btc"}
numderive = 3
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(mnemonic, coin, numderive=3)



In [25]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


In [26]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin,priv_key):
    print(coin)
    print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)



In [27]:
print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))




"0xeb6abd70a4554eb4b7c4fe51b8045060b4cb103146304b69b061276d203e19b3"
"cSRukXneF94qmFaHdrByWtAFaqqzDua7C6n5kjtzNAFWhFrb4bwu"


In [28]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1DtPs2cudZ9REuNdXFD49F2X4sJtGyXNCF",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "Kwm1xyL9EMsfyWXRgoVHdZP3PZfzxVnmEBoEZ7x3cAAwHpCYd5Ad",
            "pubkey": "032d0151a2dad5a5b2f151bdc4d7ba5394a7eec48fd441d30a563cbb1418d9809f",
            "pubkeyhash": "8d59cee3142b7390a796950a5fbf7a084264bbe8",
            "xprv": "xprvA2qAvxiyv9UAaqzNRHb4yEvEaV3MQMk2ufAyNjEDykBxKtx4pMyUpYN5DV1tfqn9zNhEZLhWDwE8NHLZk4s9fv63cc1efiTBqFLTLxjhNjw",
            "xpub": "xpub6FpXLUFskX2ToL4qXK85LNry8WsqopTtGt6aB7dqY5iwChHDMuHjNLgZ4nD8VnrP75CWXWEXHYwrCSYdH4RU8W6CpNBNHi6myV1LBtmR7yF"
        },
        {
            "address": "1LcJVVMMdgn2YpQMd5fmqAygJeZ2PnvJoJ",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "KwU3XFF1WuVZmNqWB7djrBwMiBRgeGp8y7ft3VjUssiNpCfNwi5L",
            "pubkey": "03ea48f065c3a06d041b5cebb94b4a8a3ca9c1d3046ed24d8a7e8d7cdc22a4e707",
            "pubkeyhash":

In [29]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin,account, recipient, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":eth_acc.address, "to":recipient, "value": amount}
        )
        return { 
            "from": eth_acc.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_acc.address)
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])



In [30]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin,account,recipient, amount):
    txn = create_tx(coin, account, recipient, amount)
    if coin == ETH:
        signed_txn = eth_acc.sign_transaction(txn)
        result = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
        print(result.hex())
        return result.hex()
    elif coin == BTCTEST:
        tx_btctest = create_tx(coin, account, recipient, amount)
        signed_txn = account.sign_transaction(txn)
        print(signed_txn)
        return NetworkAPI.broadcast_tx_testnet(signed_txn)

In [31]:
w3.isConnected()

False

In [32]:
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

btc-test
cSRukXneF94qmFaHdrByWtAFaqqzDua7C6n5kjtzNAFWhFrb4bwu


In [33]:
create_tx(BTCTEST,btc_acc,"mgwpqFy5NuRExe14WjFkFDdrBWH44TrdR2", 0.00001)

'{"unspents":[{"amount":95592,"confirmations":15,"script":"76a9140fac4bbe941d3803b640c558bc8c4f2497b3462f88ac","txid":"15988d8d857c517b3e507606c15b67803b6347a0bd045eaa23c032c0c7550401","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mgwpqFy5NuRExe14WjFkFDdrBWH44TrdR2",1000],["mgwpqFy5NuRExe14WjFkFDdrBWH44TrdR2",94140]]}'

In [34]:
send_tx(BTCTEST,btc_acc,"mkHS9ne12qx9pS9VojpwU5xtRd4T7X7ZUt", 0.00001)


0100000001010455c7c032c023aa5e04bda047633b80675bc10676503e7b517c858d8d9815010000006a473044022003eb50420dcb1aa9aaed29c94aac3df61d200a1a27f8d3f956c454833116dc5c02200e699380887574b34c2e08b8a613f8c2be61a729d732b3596751c109c61688b4012102405afe1499b9776b938a01c28bf0d13d9e6bcb7ee3284f6c1374352d265661eaffffffff02e8030000000000001976a914344a0f48ca150ec2b903817660b9b68b13a6702688acbc6f0100000000001976a9140fac4bbe941d3803b640c558bc8c4f2497b3462f88ac00000000


In [35]:
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [36]:
w3.isConnected()

False

In [37]:
eth_acc = priv_key_to_account(ETH, derive_wallets(mnemonic, ETH,5)[0]['privkey'])


eth
0xeb6abd70a4554eb4b7c4fe51b8045060b4cb103146304b69b061276d203e19b3
